# 🏭 IMPLEMENTANDO PIPELINE DE PRODUÇÃO

In [1]:

print("🏭 IMPLEMENTANDO PIPELINE DE PRODUÇÃO")
print("=" * 50)

# Configuração
import pandas as pd
import numpy as np
from pathlib import Path
import re
from PyPDF2 import PdfReader
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Verificar arquivos
pasta_projetos = "data/raw"
pasta_path = Path(pasta_projetos)

if not pasta_path.exists():
    caminho_absoluto = r"C:\Users\User\OneDrive - Questindustries\Projetos Python\Plataforma Quantitativos\data\raw"
    pasta_path = Path(caminho_absoluto)
    if pasta_path.exists():
        pasta_projetos = caminho_absoluto

pdfs = list(pasta_path.glob("*.pdf"))
print(f"📁 PDFs para processar: {len(pdfs)}")

🏭 IMPLEMENTANDO PIPELINE DE PRODUÇÃO
📁 PDFs para processar: 16


# 🔧 PARSER ESPECÍFICO (reutilizando do notebook 02_solucoes_avancadas.ipynb)

In [2]:

class ParserQuantitativos:
    """Parser para extrair materiais de diferentes estruturas"""
    
    def __init__(self):
        self.padroes = self._criar_padroes_especificos()
    
    def _criar_padroes_especificos(self):
        """Cria padrões baseados na análise estrutural"""
        return {
            # Padrão 1: Código - Descrição (Dimensões)
            'codigo_descricao': {
                'regex': r'(M\d{2,4})\s*[-–]?\s*([^-–]+?)\s*(?:\(([^)]+)\))?',
                'campos': ['codigo', 'descricao', 'dimensoes']
            },
            
            # Padrão 2: Item com medidas
            'item_medidas': {
                'regex': r'([^,\.]+?)\s*[,\.]?\s*(\d+)\s*x\s*(\d+)(?:\s*/\s*(\d+))?\s*cm',
                'campos': ['descricao', 'largura', 'altura', 'profundidade']
            },
            
            # Padrão 3: Quantidade + Material
            'quantidade_material': {
                'regex': r'(\d+)\s*(un|UN|pç|pc|und)?\s*[\\-]?\s*([^,\.]+)',
                'campos': ['quantidade', 'unidade', 'material']
            },
            
            # Padrão 4: Áreas
            'areas': {
                'regex': r'[Aa]\s*[=:]?\s*(\d+[,.]?\d*)\s*m²',
                'campos': ['area_m2']
            },
            
            # Padrão 5: Mobiliário completo
            'mobiliario_completo': {
                'regex': r'(M\d{3})\s*-\s*(mesa|armário|cadeira|estante)[^\d]*(\d+)[^\d]*(\d+)',
                'campos': ['codigo', 'tipo', 'dimensao1', 'dimensao2']
            }
        }
    
    def extrair_materiais_estrutura_especifica(self, texto, estrutura='codigo_descricao'):
        """Extrai materiais baseado em estrutura específica"""
        padrao = self.padroes.get(estrutura)
        if not padrao:
            return []
        
        matches = re.finditer(padrao['regex'], texto, re.IGNORECASE)
        materiais = []
        
        for match in matches:
            material = {'estrutura': estrutura}
            for i, campo in enumerate(padrao['campos']):
                if i < len(match.groups()):
                    material[campo] = match.group(i+1)
                else:
                    material[campo] = None
            material['texto_original'] = match.group(0)
            materiais.append(material)
        
        return materiais
    
    def analisar_multiplas_estruturas(self, texto):
        """Tenta múltiplas estruturas e retorna a que encontrar mais matches"""
        resultados = {}
        
        for estrutura in self.padroes.keys():
            materiais = self.extrair_materiais_estrutura_especifica(texto, estrutura)
            if materiais:
                resultados[estrutura] = {
                    'quantidade': len(materiais),
                    'materiais': materiais
                }
        
        return resultados

# 🏭 PIPELINE PRINCIPAL DE EXTRAÇÃO

In [6]:
# 🏭 PIPELINE PRINCIPAL DE EXTRAÇÃO

class PipelineExtracaoMateriais:
    """Pipeline completo para extração de materiais de projetos"""
    
    def __init__(self):
        self.parser = ParserQuantitativos()
        self.resultados_globais = {}
        self.estatisticas = {}
        
    def extrair_texto_pdf(self, pdf_path, max_paginas=None):
        """Extrai texto do PDF de forma robusta"""
        try:
            reader = PdfReader(pdf_path)
            texto = ""
            paginas = reader.pages[:max_paginas] if max_paginas else reader.pages
            
            for i, pagina in enumerate(paginas):
                texto_pagina = pagina.extract_text()
                if texto_pagina:
                    texto += f"--- PÁGINA {i+1} ---\n{texto_pagina}\n\n"
            
            return texto.strip()
        except Exception as e:
            print(f"❌ Erro ao extrair texto de {pdf_path.name}: {e}")
            return ""
    
    def identificar_secoes_relevantes(self, texto):
        """Identifica seções com materiais"""
        secoes_chave = [
            'quantitativo', 'material', 'especificação', 'descrição',
            'qtd', 'unidade', 'medidas', 'dimensões', 'mobiliário',
            'lista de materiais', 'composição', 'memorial', 'cronograma'
        ]
        
        linhas = texto.split('\n')
        secoes_encontradas = []
        
        for i, linha in enumerate(linhas):
            linha_limpa = linha.strip().lower()
            for secao in secoes_chave:
                if secao in linha_limpa:
                    secoes_encontradas.append({
                        'linha': i + 1,
                        'secao': secao,
                        'texto': linha.strip()
                    })
                    break
        
        return secoes_encontradas
    
    def processar_pdf(self, pdf_path, estrategia='auto'):
        """Processa um PDF individual"""
        print(f"\n🔄 Processando: {pdf_path.name}")
        
        # Extrair texto
        texto = self.extrair_texto_pdf(pdf_path)
        if not texto:
            return {'status': 'erro', 'mensagem': 'Texto não extraído'}
        
        # Estatísticas básicas
        linhas = texto.split('\n')
        palavras = texto.split()
        
        # Aplicar diferentes estratégias
        resultados = {}
        
        # ESTRATÉGIA 1: Parser específico em todo o texto
        resultados['parser_geral'] = self.parser.analisar_multiplas_estruturas(texto)
        
        # ESTRATÉGIA 2: Processamento por seções
        secoes = self.identificar_secoes_relevantes(texto)
        resultados['secoes_identificadas'] = secoes
        
        if secoes:
            # Extrair materiais das seções relevantes
            linhas_texto = texto.split('\n')
            materiais_secoes = {}
            
            for secao in secoes[:5]:  # Limitar a 5 seções para performance
                inicio = max(0, secao['linha'] - 3)
                fim = min(len(linhas_texto), secao['linha'] + 15)
                texto_secao = '\n'.join(linhas_texto[inicio:fim])
                
                resultados_secao = self.parser.analisar_multiplas_estruturas(texto_secao)
                if resultados_secao:
                    materiais_secoes[secao['secao']] = {
                        'range': f"{inicio+1}-{fim+1}",
                        'resultados': resultados_secao
                    }
            
            resultados['materiais_secoes'] = materiais_secoes
        
        # Consolidar resultados
        resultado_final = {
            'status': 'sucesso',
            'arquivo': pdf_path.name,
            'estatisticas_texto': {
                'linhas': len(linhas),
                'palavras': len(palavras),
                'caracteres': len(texto)
            },
            'secoes_identificadas': len(secoes),
            'resultados': resultados
        }
        
        return resultado_final
    
    def executar_pipeline_lote(self, pdfs, max_pdfs=None):
        """Executa pipeline em lote para múltiplos PDFs"""
        print(f"🏭 INICIANDO PIPELINE EM LOTE")
        print(f"📁 Total de PDFs: {len(pdfs)}")
        if max_pdfs:
            pdfs = pdfs[:max_pdfs]
            print(f"🔢 Limitando a: {max_pdfs} PDFs")
        
        resultados_lote = {}
        
        for i, pdf_path in enumerate(pdfs, 1):
            print(f"\n[{i}/{len(pdfs)}] Processando: {pdf_path.name}")
            
            try:
                resultado = self.processar_pdf(pdf_path)
                resultados_lote[pdf_path.name] = resultado
                
                # Estatísticas em tempo real
                if resultado['status'] == 'sucesso':
                    total_materiais = self._calcular_total_materiais(resultado)
                    print(f"   ✅ Sucesso - {total_materiais} materiais encontrados")
                else:
                    print(f"   ❌ {resultado['mensagem']}")
                    
            except Exception as e:
                print(f"   💥 Erro crítico: {e}")
                resultados_lote[pdf_path.name] = {
                    'status': 'erro_critico',
                    'mensagem': str(e)
                }
        
        # Análise consolidada
        self._gerar_relatorio_consolidado(resultados_lote)
        
        return resultados_lote
    
    def _calcular_total_materiais(self, resultado):
        """Calcula total de materiais encontrados"""
        total = 0
        
        # Materiais do parser geral
        if 'parser_geral' in resultado['resultados']:
            for estrutura, dados in resultado['resultados']['parser_geral'].items():
                total += dados['quantidade']
        
        # Materiais das seções
        if 'materiais_secoes' in resultado['resultados']:
            for secao, dados in resultado['resultados']['materiais_secoes'].items():
                for estrutura, resultados_estrutura in dados['resultados'].items():
                    total += resultados_estrutura['quantidade']
        
        return total
    
    def _gerar_relatorio_consolidado(self, resultados_lote):
        """Gera relatório consolidado do processamento"""
        print(f"\n" + "="*60)
        print("📊 RELATÓRIO CONSOLIDADO DO PIPELINE")
        print("="*60)
        
        total_pdfs = len(resultados_lote)
        pdfs_sucesso = [r for r in resultados_lote.values() if r['status'] == 'sucesso']
        pdfs_erro = [r for r in resultados_lote.values() if r['status'] != 'sucesso']
        
        print(f"📁 Total processado: {total_pdfs} PDFs")
        print(f"✅ Com sucesso: {len(pdfs_sucesso)} PDFs")
        print(f"❌ Com erro: {len(pdfs_erro)} PDFs")
        
        if pdfs_sucesso:
            # Estatísticas de materiais
            totais_materiais = []
            for resultado in pdfs_sucesso:
                total = self._calcular_total_materiais(resultado)
                totais_materiais.append(total)
            
            print(f"\n🏗️  ESTATÍSTICAS DE MATERIAIS:")
            print(f"   • Total de materiais extraídos: {sum(totais_materiais)}")
            print(f"   • Média por PDF: {np.mean(totais_materiais):.1f}")
            print(f"   • PDF com mais materiais: {max(totais_materiais)}")
            print(f"   • PDF com menos materiais: {min(totais_materiais)}")
            
            # PDFs com melhor performance
            print(f"\n🏆 TOP 3 PDFs COM MAIS MATERIAIS:")
            performance_pdfs = []
            for nome, resultado in resultados_lote.items():
                if resultado['status'] == 'sucesso':
                    total = self._calcular_total_materiais(resultado)
                    performance_pdfs.append((nome, total))
            
            for nome, total in sorted(performance_pdfs, key=lambda x: x[1], reverse=True)[:3]:
                print(f"   📊 {nome}: {total} materiais")
        
        # Salvar resultados
        self._salvar_resultados(resultados_lote)
    
    def _salvar_resultados(self, resultados_lote):
        """Salva resultados em arquivos"""
        import os
        
        # Criar pasta processed se não existir
        pasta_processed = Path("../data/processed")
        pasta_processed.mkdir(parents=True, exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Salvar JSON completo
        arquivo_json = pasta_processed / f"resultados_pipeline_{timestamp}.json"
        with open(arquivo_json, 'w', encoding='utf-8') as f:
            json.dump(resultados_lote, f, indent=2, ensure_ascii=False)
        print(f"💾 Resultados salvos em: {arquivo_json}")
        
        # Salvar CSV resumido
        self._exportar_csv(resultados_lote, timestamp)
    
    def _exportar_csv(self, resultados_lote, timestamp):
        """Exporta resultados resumidos para CSV"""
        import os
        
        pasta_processed = Path("../data/processed")
        pasta_processed.mkdir(parents=True, exist_ok=True)
        
        dados_csv = []
        
        for nome_arquivo, resultado in resultados_lote.items():
            if resultado['status'] == 'sucesso':
                linha = {
                    'arquivo': nome_arquivo,
                    'status': 'sucesso',
                    'total_materiais': self._calcular_total_materiais(resultado),
                    'linhas_texto': resultado['estatisticas_texto']['linhas'],
                    'palavras_texto': resultado['estatisticas_texto']['palavras'],
                    'secoes_identificadas': resultado['secoes_identificadas']
                }
                dados_csv.append(linha)
            else:
                linha = {
                    'arquivo': nome_arquivo,
                    'status': 'erro',
                    'total_materiais': 0,
                    'linhas_texto': 0,
                    'palavras_texto': 0,
                    'secoes_identificadas': 0
                }
                dados_csv.append(linha)
        
        df = pd.DataFrame(dados_csv)
        arquivo_csv = pasta_processed / f"resultados_resumido_{timestamp}.csv"
        df.to_csv(arquivo_csv, index=False, encoding='utf-8')
        print(f"📊 CSV resumido salvo em: {arquivo_csv}")
        
        return df

# Instanciar o pipeline corrigido
pipeline = PipelineExtracaoMateriais()

# 🧪 EXECUTAR PIPELINE EM MODO TESTE

In [7]:
print("🧪 EXECUTANDO PIPELINE EM MODO TESTE")
print("=" * 45)

# Executar em 3 PDFs para teste
resultados_teste = pipeline.executar_pipeline_lote(pdfs[:3])

print(f"\n✅ PIPELINE DE TESTE CONCLUÍDO!")

🧪 EXECUTANDO PIPELINE EM MODO TESTE
🏭 INICIANDO PIPELINE EM LOTE
📁 Total de PDFs: 3

[1/3] Processando: Projeto arquitetura Legal Cartão Joinville.pdf

🔄 Processando: Projeto arquitetura Legal Cartão Joinville.pdf


Multiple definitions in dictionary at byte 0x674b3 for key /PageMode


   ✅ Sucesso - 413 materiais encontrados

[2/3] Processando: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf

🔄 Processando: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf
   ✅ Sucesso - 169 materiais encontrados

[3/3] Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf

🔄 Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf
   ✅ Sucesso - 105 materiais encontrados

📊 RELATÓRIO CONSOLIDADO DO PIPELINE
📁 Total processado: 3 PDFs
✅ Com sucesso: 3 PDFs
❌ Com erro: 0 PDFs

🏗️  ESTATÍSTICAS DE MATERIAIS:
   • Total de materiais extraídos: 687
   • Média por PDF: 229.0
   • PDF com mais materiais: 413
   • PDF com menos materiais: 105

🏆 TOP 3 PDFs COM MAIS MATERIAIS:
   📊 Projeto arquitetura Legal Cartão Joinville.pdf: 413 materiais
   📊 PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf: 169 materiais
   📊 PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf: 105 materiais
💾 Resultados salvos em: ..\data\processed\resultados_pipeline_20251017_175208.json


# 💾 SISTEMA DE EXPORTAÇÃO E RELATÓRIOS

In [8]:

class ExportadorResultados:
    """Exporta resultados em formatos estruturados"""
    
    def __init__(self, resultados):
        self.resultados = resultados
    
    def exportar_materiais_detalhados(self, arquivo_saida):
        """Exporta todos os materiais encontrados em formato estruturado"""
        print(f"\n💾 Exportando materiais detalhados...")
        
        # Criar pasta se não existir
        arquivo_saida_path = Path(arquivo_saida)
        arquivo_saida_path.parent.mkdir(parents=True, exist_ok=True)
        
        todos_materiais = []
        
        for nome_arquivo, resultado in self.resultados.items():
            if resultado['status'] == 'sucesso':
                # Materiais do parser geral
                if 'parser_geral' in resultado['resultados']:
                    for estrutura, dados in resultado['resultados']['parser_geral'].items():
                        for material in dados['materiais']:
                            material_export = {
                                'arquivo': nome_arquivo,
                                'estrutura': estrutura,
                                'texto_original': material['texto_original'],
                                'campos': material
                            }
                            todos_materiais.append(material_export)
        
        # Criar DataFrame
        if todos_materiais:
            df_materiais = pd.DataFrame(todos_materiais)
            
            # Expandir campos
            campos_expandidos = []
            for material in todos_materiais:
                linha = {
                    'arquivo': material['arquivo'],
                    'estrutura': material['estrutura'],
                    'texto_original': material['texto_original']
                }
                # Adicionar campos específicos
                for campo, valor in material['campos'].items():
                    if campo not in ['estrutura', 'texto_original']:
                        linha[campo] = valor
                campos_expandidos.append(linha)
            
            df_final = pd.DataFrame(campos_expandidos)
            df_final.to_csv(arquivo_saida, index=False, encoding='utf-8')
            print(f"✅ Materiais exportados: {len(df_final)} itens")
            print(f"📊 Arquivo: {arquivo_saida}")
            
            return df_final
        else:
            print("⚠️  Nenhum material para exportar")
            return pd.DataFrame()
    
    def gerar_relatorio_executivo(self):
        """Gera relatório executivo em texto"""
        print(f"\n📋 RELATÓRIO EXECUTIVO")
        print("=" * 40)
        
        total_pdfs = len(self.resultados)
        pdfs_sucesso = [r for r in self.resultados.values() if r['status'] == 'sucesso']
        pdfs_erro = [r for r in self.resultados.values() if r['status'] != 'sucesso']
        
        # Calcular totais
        totais_materiais = []
        for resultado in pdfs_sucesso:
            total = 0
            if 'parser_geral' in resultado['resultados']:
                for estrutura, dados in resultado['resultados']['parser_geral'].items():
                    total += dados['quantidade']
            totais_materiais.append(total)
        
        print(f"🏭 RESUMO DO PROCESSAMENTO:")
        print(f"   • PDFs processados: {total_pdfs}")
        print(f"   • Processamentos bem-sucedidos: {len(pdfs_sucesso)}")
        print(f"   • Processamentos com erro: {len(pdfs_erro)}")
        
        if pdfs_sucesso:
            print(f"\n🏗️  ESTATÍSTICAS DE MATERIAIS:")
            print(f"   • Total de itens extraídos: {sum(totais_materiais)}")
            print(f"   • Média por projeto: {np.mean(totais_materiais):.1f} itens")
            print(f"   • Maior extração: {max(totais_materiais)} itens")
            print(f"   • Menor extração: {min(totais_materiais)} itens")
            
            # Top projetos
            print(f"\n🏆 PROJETOS COM MELHOR RESULTADO:")
            performance = []
            for nome, resultado in self.resultados.items():
                if resultado['status'] == 'sucesso':
                    total = 0
                    if 'parser_geral' in resultado['resultados']:
                        for estrutura, dados in resultado['resultados']['parser_geral'].items():
                            total += dados['quantidade']
                    performance.append((nome, total))
            
            for nome, total in sorted(performance, key=lambda x: x[1], reverse=True)[:3]:
                print(f"   🥇 {nome}: {total} materiais")
        
        print(f"\n💡 RECOMENDAÇÕES:")
        if len(pdfs_erro) > len(pdfs_sucesso):
            print("   ⚠️  Muitos erros - revisar qualidade dos PDFs")
        elif sum(totais_materiais) == 0:
            print("   🔴 Nenhum material extraído - ajustar padrões regex")
        else:
            print("   ✅ Pipeline funcionando adequadamente")
            print("   → Considerar implementar OCR para PDFs scanneds")
            print("   → Expandir para mais tipos de projetos")

# 🏭 EXECUTAR PIPELINE COMPLETO

In [9]:

print("\n" + "="*70)
print("🏭 INICIANDO PIPELINE COMPLETO - TODOS OS PDFs")
print("="*70)

# Executar pipeline em todos os PDFs
resultados_completos = pipeline.executar_pipeline_lote(pdfs)

print(f"\n🎉 PIPELINE COMPLETO CONCLUÍDO!")


🏭 INICIANDO PIPELINE COMPLETO - TODOS OS PDFs
🏭 INICIANDO PIPELINE EM LOTE
📁 Total de PDFs: 16

[1/16] Processando: Projeto arquitetura Legal Cartão Joinville.pdf

🔄 Processando: Projeto arquitetura Legal Cartão Joinville.pdf


Multiple definitions in dictionary at byte 0x674b3 for key /PageMode


   ✅ Sucesso - 413 materiais encontrados

[2/16] Processando: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf

🔄 Processando: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf
   ✅ Sucesso - 169 materiais encontrados

[3/16] Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf

🔄 Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout1.pdf
   ✅ Sucesso - 105 materiais encontrados

[4/16] Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout2.pdf

🔄 Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout2.pdf
   ✅ Sucesso - 61 materiais encontrados

[5/16] Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout3.pdf

🔄 Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout3.pdf
   ✅ Sucesso - 59 materiais encontrados

[6/16] Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout4.pdf

🔄 Processando: PROJ_ARQ_LEGAL_AmorSaúde_São Mateus_SP_R04-Layout4.pdf


Multiple definitions in dictionary at byte 0x1a8527 for key /PageMode


   ✅ Sucesso - 64 materiais encontrados

[7/16] Processando: RA-BVAR-2402205-01_CAMBUÍ_ARQ_EXE_R00_OP02-CIVIL.pdf

🔄 Processando: RA-BVAR-2402205-01_CAMBUÍ_ARQ_EXE_R00_OP02-CIVIL.pdf


...

   ✅ Sucesso - 63 materiais encontrados

[10/16] Processando: RA-BVAR-2402205-01_CAMBUÍ_ARQ_EXE_R00_OP02-PISO.pdf

🔄 Processando: RA-BVAR-2402205-01_CAMBUÍ_ARQ_EXE_R00_OP02-PISO.pdf
   ✅ Sucesso - 145 materiais encontrados

[11/16] Processando: RF_SETE_LAGOAS_EXECUTIVO_R02.pdf

🔄 Processando: RF_SETE_LAGOAS_EXECUTIVO_R02.pdf


Multiple definitions in dictionary at byte 0xd16be for key /PageMode


   ✅ Sucesso - 236 materiais encontrados

[12/16] Processando: RF_SETE_LAGOAS_FACHADA_R01.pdf

🔄 Processando: RF_SETE_LAGOAS_FACHADA_R01.pdf
   ✅ Sucesso - 76 materiais encontrados

[13/16] Processando: RF_SETE_LAGOAS_MOBILIARIO_R00.pdf

🔄 Processando: RF_SETE_LAGOAS_MOBILIARIO_R00.pdf


Multiple definitions in dictionary at byte 0x10924b for key /PageMode


   ✅ Sucesso - 458 materiais encontrados

[14/16] Processando: RF_VIAMA_O_FACHADA_R03.pdf

🔄 Processando: RF_VIAMA_O_FACHADA_R03.pdf
   ✅ Sucesso - 82 materiais encontrados

[15/16] Processando: RF_VIAMA_O_MOBILIARIO_R01.pdf

🔄 Processando: RF_VIAMA_O_MOBILIARIO_R01.pdf
   ✅ Sucesso - 510 materiais encontrados

[16/16] Processando: RF_VIAMÃO_EXECUTIVO_R01.pdf

🔄 Processando: RF_VIAMÃO_EXECUTIVO_R01.pdf
   ✅ Sucesso - 243 materiais encontrados

📊 RELATÓRIO CONSOLIDADO DO PIPELINE
📁 Total processado: 16 PDFs
✅ Com sucesso: 16 PDFs
❌ Com erro: 0 PDFs

🏗️  ESTATÍSTICAS DE MATERIAIS:
   • Total de materiais extraídos: 2930
   • Média por PDF: 183.1
   • PDF com mais materiais: 510
   • PDF com menos materiais: 51

🏆 TOP 3 PDFs COM MAIS MATERIAIS:
   📊 RF_VIAMA_O_MOBILIARIO_R01.pdf: 510 materiais
   📊 RF_SETE_LAGOAS_MOBILIARIO_R00.pdf: 458 materiais
   📊 Projeto arquitetura Legal Cartão Joinville.pdf: 413 materiais
💾 Resultados salvos em: ..\data\processed\resultados_pipeline_20251017_175728

# 💾 EXPORTAR RESULTADOS

In [10]:
# Exportar resultados
if 'resultados_completos' in locals() and resultados_completos:
    exportador = ExportadorResultados(resultados_completos)
    
    # Exportar materiais detalhados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    arquivo_materiais = f"../data/processed/materiais_detalhados_{timestamp}.csv"
    df_materiais = exportador.exportar_materiais_detalhados(arquivo_materiais)
    
    # Gerar relatório executivo
    exportador.gerar_relatorio_executivo()

print(f"\n🎯 PRÓXIMAS ETAPAS:")
print("   1. Analisar qualidade da extração manualmente")
print("   2. Refinar padrões regex baseado nos resultados") 
print("   3. Implementar sistema de validação")
print("   4. Desenvolver interface web")
print("   5. Integrar com banco de dados")

print(f"\n🚀 PIPELINE DE PRODUÇÃO IMPLEMENTADO COM SUCESSO!")


💾 Exportando materiais detalhados...
✅ Materiais exportados: 2778 itens
📊 Arquivo: ../data/processed/materiais_detalhados_20251017_180152.csv

📋 RELATÓRIO EXECUTIVO
🏭 RESUMO DO PROCESSAMENTO:
   • PDFs processados: 16
   • Processamentos bem-sucedidos: 16
   • Processamentos com erro: 0

🏗️  ESTATÍSTICAS DE MATERIAIS:
   • Total de itens extraídos: 2778
   • Média por projeto: 173.6 itens
   • Maior extração: 508 itens
   • Menor extração: 49 itens

🏆 PROJETOS COM MELHOR RESULTADO:
   🥇 RF_VIAMA_O_MOBILIARIO_R01.pdf: 508 materiais
   🥇 RF_SETE_LAGOAS_MOBILIARIO_R00.pdf: 456 materiais
   🥇 Projeto arquitetura Legal Cartão Joinville.pdf: 331 materiais

💡 RECOMENDAÇÕES:
   ✅ Pipeline funcionando adequadamente
   → Considerar implementar OCR para PDFs scanneds
   → Expandir para mais tipos de projetos

🎯 PRÓXIMAS ETAPAS:
   1. Analisar qualidade da extração manualmente
   2. Refinar padrões regex baseado nos resultados
   3. Implementar sistema de validação
   4. Desenvolver interface web